# Início Colab

# Importações

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from math import ceil

import torch
from torch import nn
from torchvision import transforms, datasets, models, utils
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import functional as VF

# Utilidades

Função para mostrar uma imagem.
*   Recebe um tensor de imagem e a sua label correspondente
*   Mostra a imagem dada com a label de título

In [ ]:
def plot_img(img, label = 'NULL'):
    _,ax1 = plt.subplots(1)
    img = VF.to_pil_image(img[0])
    show = ax1.imshow(img)
    
    ax1.set_title(label)
    
    plt.show()

Função para mostrar dois gráficos.


*   Gráfico das losses de teste e validação durante as epochs
*   Gráfico da acurácia durante as epochs



In [ ]:
def plot_train_graph(list_train_loss, list_eval_loss, list_acc, epochs):
    fig, axs = plt.subplots(2, figsize=(15,8))
    axs[0].plot(range(1, epochs+1), list_train_loss,label="Train loss") #Plota o erro de treino X epocas
    axs[0].plot(range(1, epochs+1), list_eval_loss, label="Test loss") #Plota o erro de teste x epocas
    axs[0].set(xticks=range(1, epochs+1))
    axs[0].set_title('Losses x Epochs') #Define o título
    axs[0].set(xlabel='Epochs') #Define o nome do eixo x
    axs[0].set(ylabel='Losses') #Define o nome do eixo y
    axs[0].legend() #Mostra a legenda
    axs[0].grid(True) #Mostra a grade


    axs[1].plot(range(1, epochs+1), list_acc, label='Accuracy') #Plota a acuracia para erro de 1 graus x epocas
    axs[1].set(xticks=range(1, epochs+1))
    axs[1].set_title('Accuracy x epochs') #Define o título
    axs[1].set(xlabel='Epochs') #Define o nome do eixo x
    axs[1].set(ylabel='Accuracy (%)') #Define o nome do eixo y
    axs[1].legend(loc='lower right') #Mostra a legenda
    axs[1].grid(True) #Mostra a grade

    plt.tight_layout()

# Processamento dos dados

Lê os dados usando torchvision.
*   transforms faz modificações nas fotos por fins de desempenho (data augmentation)
*   classes é uma lista com todas as labels ordenadas pela classe

In [ ]:
train_transforms = transforms.Compose(
    [transforms.Resize((224,224)),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    transforms.ToTensor(),
    ])

raw_train_data = datasets.ImageFolder("dataset/image",
                                      transform = train_transforms,
                                      )

In [ ]:
classes = list((raw_train_data.class_to_idx).keys())
classes.append('10 - Inconclusivo')

Separa o dataset em treino, validação e teste.

In [ ]:
train_percentage = 75
eval_percentage = 20
test_percentage = 5

n_train_data = len(raw_train_data)

n_train = round(n_train_data * (train_percentage/100))
n_eval = round(n_train_data * (eval_percentage/100))

n_test = round(n_train_data * (test_percentage/100))

if(n_train + n_eval + n_test != n_train_data):
    n_train = n_train_data - n_eval - n_test


print(f'train: {n_train} eval: {n_eval} test: {n_test} total: {n_train_data}')

Faz a separação de forma aleatória

In [ ]:
train_data_unloaded, eval_data_unloaded, test_data_unloaded = random_split(raw_train_data, [n_train, n_eval, n_test])

Cria os dataLoaders de teste, validação e treino

In [ ]:
batch_size = 25

train_data = DataLoader(train_data_unloaded, batch_size=batch_size, shuffle=True,)
eval_data = DataLoader(eval_data_unloaded, batch_size=1, shuffle=True,)
test_data = DataLoader(test_data_unloaded, batch_size=1, shuffle=True)

# Exemplo com foto qualquer

Mostra uma imagem qualquer do dataset de treino

In [ ]:
img, labels = next(iter(test_data))
plot_img(img, (classes[labels])[4:])

# Definição do modelo

Decidimos utilizar da técnica de transfer learning


*   Modelo pré-treinado: resnext101_32x8d



In [ ]:
model = models.resnext101_32x8d(pretrained = True)
for param in model.parameters():
    param.requires_grad = False

Treinando na gpu ou cpu?

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device.type

Definição das FCL


*   TODO
*   O modelo é enviado para a gpu (caso tenha cuda)



In [ ]:
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
    )

model = model.to(device)



*   TODO
*   Otimizador escolhido é o AdamW com learning rate padrão (1e-3)



In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.fc.parameters())

# Treinamento

Definição da função de treino


*   A função define somente uma época
*   retorna as losses e a acurácia obtida na validação



In [ ]:
norm = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])

In [ ]:
def train(model, train_data, eval_data):
    
    # Train steps
    
    train_loss = 0
    eval_loss = 0
    acc = 0

    model.train()
    for images, labels in train_data:
        images = norm(images)

        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = criterion(output, labels)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        train_loss += loss
    
    train_loss /= len(train_data)

    # Validation steps
    
    n_prev = 0
    correct = 0
    
    model.eval()
    with torch.no_grad():
        for v_images, v_labels in eval_data:
            v_images = norm(v_images)

            v_images = v_images.to(device)
            v_labels = v_labels.to(device)

            v_output = model(v_images)
            eval_loss += criterion(v_output, v_labels)

            val,prev = torch.max(v_output, dim = 1)
            n_prev += 1

            if prev == v_labels and val > 0.75:
                correct += 1
        
        eval_loss /= len(eval_data)
        
        acc = correct/n_prev*100
    
    return train_loss, eval_loss, acc

Etapa de treinamento


*   O número de épocas é definida na variavel epochs
*   ao fim de cada época é imprimido na tela as losses e a acurácia obtida

In [ ]:
epochs = 10

list_train_loss = []
list_eval_loss = []
list_acc = []

for epoch in range(1, epochs+1):
    train_loss, eval_loss, acc = train(model, train_data, eval_data)

    list_train_loss.append(train_loss)
    list_eval_loss.append(eval_loss)
    list_acc.append(acc)

    if acc >= max(list_acc):
        #Comente para não sobrescrever o modelo bom
        torch.save(model, 'model.pt')

    print(f'Epoch: {epoch} loss: {train_loss.item():.4f} validation loss: {eval_loss.item():.4f} Accuracy: {acc:.2f}%')

Mostra os gráficos do treinamento

In [ ]:
plot_train_graph(list_train_loss, list_eval_loss, list_acc, epochs)
model = torch.load('model.pt')

# Teste



Etapa de teste

*   envia o dataset de treino para uma rede já treinada
*   recebe a acurácia do teste

In [ ]:
wrong_img = []
def test(model, test_data):
    n_prev = 0
    correct = 0
    
    model.eval()
    with torch.no_grad():
        for t_images, t_labels in test_data:
            aux_img = t_images.to(device)

            t_images = norm(t_images)
        
            t_images = t_images.to(device)
            t_labels = t_labels.to(device)

            t_output = model(t_images)

            value,prev = torch.max(nn.functional.softmax(t_output, dim=1), dim = 1)
            value *= 100
            n_prev += 1

            if prev == t_labels and value > 75:
                correct += 1
            else:
                wrong_img.append((aux_img, prev, t_labels, value.item()))
    
    return correct/n_prev*100

Imprime a acurácia obtida

In [ ]:
test_acc = test(model, test_data)
print(f'Accuracy: {test_acc:.2f}')

# Predict

In [ ]:
def prep_img(img):
    test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                          transforms.ToTensor()])
    img = test_transforms(img)
    return img[None]

In [ ]:
def predict(img, model):
    test_out = 0
    value = 0
    prev = 0
    class_ = ''

    img = norm(img)
    if torch.cuda.is_available():
        img = img.cuda()

    with torch.no_grad():
        test_out = model(img)
        value,prev = torch.max(torch.nn.functional.softmax(test_out, dim=1), dim = 1)

    value *= 100

    if value > 75:
        if prev < 5:
            class_ = 'a'
        else:
            class_ = 'b'
    else:
        prev = 10
        class_ = 'c'

    return prev, class_

In [ ]:
img, label = next(iter(test_data))

prev, class_ = predict(img, model)

print(f'Prediction: {(classes[prev])[4:]}\nClass: {class_}')
plot_img(img, (classes[label])[4:])

for img, prev, label, value in wrong_img:
    print(f'\nPrediction: {classes[prev][4:]}\nAcc: {value:.0f}')
    plot_img(img, classes[label][4:])

# Testar com imagem da Net

In [ ]:
from PIL import Image
import requests

url = 'https://2.bp.blogspot.com/-zBIj5pr_Kts/Wn8Bbh2hGZI/AAAAAAAAT3s/bStHwEpokOopUJmsEt0IgtlE-mZS-MtaQCLcBGAs/s1600/tmana.jpg'

with Image.open(requests.get(url, stream=True).raw).convert('RGB') as img:
    img = prep_img(img)
    plot_img(img)
    prediction, class_net = predict(img, model)

print(f'Prediction: {classes[prediction][4:]}\nClass: {class_net}')